This tutorial will cover how to run an EASIER-net model, with a regression and classification example. 

We will also how you can integrate GridSearchCV from scikit-learn.

In [ ]:
import numpy as np
from easier_net import EasierNetEstimator
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

#### Regression

In [ ]:
# Make some fake data
def make_data(n_samples, n_features=20):
    x = np.random.rand(n_samples,n_features)
    beta = np.ones(n_features).reshape((-1,1))
    eps = np.random.randn(n_samples) * 0.1
    y = x @ beta + eps
    return x,y

np.random.seed(0)
train_x, train_y = make_data(1000)
test_x, test_y = make_data(1000)

In [ ]:
# Train an easier net
e_net = EasierNetEstimator(
    n_estimators=2,
    input_filter_layer=True,
    n_layers=3,
    n_hidden=20,
    full_tree_pen=0.1,
    input_pen=0.01,
    batch_size=100,
    num_classes=0,
    weight=[],
    max_iters=800,
    max_prox_iters=20)

e_net.fit(train_x, train_y)

In [ ]:
# Evaluate on the test data
predictions = e_net.predict(test_x)
mse = np.mean(np.power(predictions - test_y, 2))
print(mse)

##### Grid Search for Regression

In [ ]:
# Perform GridSearchCV
# Set parameter grid
param_grid = [
    {'n_estimators': [5], 'n_layers': [5], 'n_hidden': [100], 'input_pen': [0.01, 0.1, 1], 
     'full_tree_pen': [0.01, 0.1, 1], 'max_iters': [100], 'max_prox_iters': [50]},
]

e_net_gridsearch = GridSearchCV(
    estimator = e_net, 
    param_grid = param_grid, 
    cv = 5).fit(train_x, train_y)

# Return predictions with the estimator that has the best parameters
y_pred = e_net_gridsearch.predict(test_x)

# Return score on given test data
e_net_gridsearch.score(test_x, test_y)

#### Classification

In [12]:
def make_class_data(n_samples, num_classes, n_features=20): 
    x = np.random.rand(n_samples,n_features)
    beta = np.zeros(n_features)
    beta[:n_features//2] = 1
    prob_y = 1/(1 + np.exp(-np.matmul(x, beta)))
    y = np.random.binomial(n=1, p=prob_y).reshape((-1,1))
    return x,y

np.random.seed(0)
cls_train_x, cls_train_y = make_class_data(1000, 2)
cls_test_x, cls_test_y = make_class_data(1000, 2)

In [15]:
# Train an easier net
cls_easier_net = EasierNetEstimator(
    n_estimators=2,
    input_filter_layer=True,
    n_layers=3,
    n_hidden=20,
    full_tree_pen=0.1,
    input_pen=0.01,
    batch_size=100,
    num_classes=2, #set this to however many class you have in your dataset
    weight=[1,1],
    max_iters=800,
    max_prox_iters=20)

cls_easier_net.fit(cls_train_x, cls_train_y)

[0] loss: 19.856063842773438 empirical: 0.6230040788650513
[100] loss: 0.2376309633255005 empirical: 0.07095792144536972
[200] loss: 0.20347991585731506 empirical: 0.10476058721542358
[300] loss: 0.20324358344078064 empirical: 0.1298779845237732
[400] loss: 0.15985345840454102 empirical: 0.09884363412857056
[500] loss: 0.12062814831733704 empirical: 0.0608196035027504
[600] loss: 0.08263266086578369 empirical: 0.023281900212168694
[700] loss: 0.1586335301399231 empirical: 0.09824790805578232
[prox 0] loss: 0.10962861776351929 empirical: 0.07200349867343903
[0] loss: 19.381196975708008 empirical: 0.7526767253875732
[100] loss: 0.2602955996990204 empirical: 0.07801123708486557
[200] loss: 0.18761777877807617 empirical: 0.06696801632642746
[300] loss: 0.13778820633888245 empirical: 0.059880439192056656
[400] loss: 0.135111466050148 empirical: 0.06188954412937164
[500] loss: 0.19520701467990875 empirical: 0.13544189929962158
[600] loss: 0.08348388969898224 empirical: 0.02335783652961254
[7

In [16]:
# Evaluate on the test data
predict_log_proba = np.log(cls_easier_net.predict_proba(cls_test_x))
neg_log_lik = -np.mean(predict_log_proba[np.arange(cls_test_y.size),cls_test_y.flatten()])
print(neg_log_lik)

0.049476746


##### Grid Search for Classification

In [ ]:
# Perform GridSearchCV
# Set parameter grid
param_grid = [
    {'n_estimators': [5], 'n_layers': [5], 'n_hidden': [100], 'input_pen': [0.01, 0.1, 1], 
     'full_tree_pen': [0.01, 0.1, 1], 'max_iters': [100], 'max_prox_iters': [50]},
]

cls_e_net_gridsearch = GridSearchCV(
    estimator = cls_easier_net, 
    param_grid = param_grid, 
    cv = 5).fit(cls_train_x, cls_train_y)

# Return predicted log probabilities for given test data
y_pred_log_proba = cls_e_net_gridsearch.predict_log_proba(cls_test_x)

# Return score on given test data
cls_e_net_gridsearch.score(cls_test_x, cls_test_y)
